# Classifying Traffic Signs with OpenCV

For this project we will be trying split the traffic sign dataset into 3 separate classes:
- Stop signs
- Red Circles
- Blue rectangles/squares 

### Imports

In [29]:
import cv2 as cv
import numpy as np
import os, os.path

### Useful functions

In [30]:
def get_file_list():
    file_list = []
    for root, _, files in os.walk('./Dataset'):
	    for file in files:
		    file_list.append(os.path.join(root,file))
    return file_list

In [31]:
def process_image(img, color):
    img = smooth(img)
    if color == "blue":
        kernel = np.ones((7,7), np.uint8)
        img = cv.dilate(img, kernel, iterations=3)
        img = cv.erode(img, kernel, iterations=3)
    img = cv.Canny(img, 230, 254) 
    final, prediction = search_contours(img, color)
    
    return final, prediction

In [32]:
# Remove image noise with gaussian blur (better at preserving edges)
def smooth(img):
    return cv.GaussianBlur(img, (3,3), 0)


In [33]:
#  Segments reds and blues of an image
def segment(img, color):
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    if color == 'red':
        # lower mask (0-10)
        lower_red = np.array([0,90,50])
        upper_red = np.array([10,255,255])
        mask0 = cv.inRange(img_hsv, lower_red, upper_red)  

        # upper mask (170-180)
        lower_red = np.array([170,90,50])
        upper_red = np.array([180,255,255])
        mask1 = cv.inRange(img_hsv, lower_red, upper_red)

        mask = mask0 | mask1
    elif color == 'blue':
        lower_blue = np.array([100,160,80])
        upper_blue = np.array([130,255,170])
        mask = cv.inRange(img_hsv, lower_blue, upper_blue) 

    segmented_img = cv.bitwise_and(img_hsv, img_hsv, mask = mask)
    segmented_img = cv.cvtColor(segmented_img, cv.COLOR_BGR2GRAY)
    return segmented_img    


In [34]:
# Find the contour with the largest area in a gray scale image
def find_largest_contour(img_blurred):
    contours, _ = cv.findContours(img_blurred,2,1)
    cnt = contours
    big_contour = []
    max = 0
    for i in cnt:
        area = cv.contourArea(i)
        if(area > max):
            max = area
            big_contour = i 
    if max > 1000 and len(big_contour) > 100 and len(big_contour) < 1200:
        final = cv.drawContours(img, big_contour, -1, (0,255,0), 3)
    else:
        final = img
    return final, big_contour

In [35]:
## TODO: Improve sensibility to blue squares 
# Find a blue square
def search_contours(img, color):
    if cv.countNonZero(img) == 0:
        return img, 'unrecognised'
    final, contour = find_largest_contour(img)
    if len(contour > 100):
        peri = cv.arcLength(contour, True)
        approx = cv.approxPolyDP(contour, 0.01 * peri, True)
        if color == 'blue':
            if len(approx) == 4:
                return final, 'square'
        elif color == 'red':
                # Find type of red sign
                if len(approx) == 8 :
                    return final, 'stop_sign'
                elif len(approx) > 8:
                    return final, 'red_circle'

    return final, 'unrecognised'

## Red Circles

This is the section to detect red circles

After processing the image to gray values holding as the white value the red features a contour finder is used to detect an ellipse correctly. It can also be used in the case of stop signs to obtain octagons.

In [36]:
prediction = {}
for file in get_file_list():
    img = cv.imread(file)
    img = smooth(img)
    
    img_blue, img_red = segment(img, 'blue'), segment(img, 'red')
    
    cv.imshow("filtered blue", img_blue)
    cv.imshow("filtered red", img_red)
    
    final, prediction[file] = process_image(img_blue, 'blue')
    if prediction[file] == 'unrecognised':
        final, prediction[file] = process_image(img_red, 'red')
        
    cv.imshow(prediction[file], final)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
print(prediction)
cv.destroyAllWindows()
    

{'./Dataset/red_circles/road115.png': 'red_circle', './Dataset/red_circles/road100.png': 'red_circle', './Dataset/red_circles/road106.png': 'red_circle', './Dataset/red_circles/road111.png': 'red_circle', './Dataset/red_circles/road104.png': 'red_circle', './Dataset/red_circles/road105.png': 'red_circle', './Dataset/red_circles/road102.png': 'red_circle', './Dataset/blue_squares/road143.png': 'square', './Dataset/blue_squares/road204.png': 'square', './Dataset/blue_squares/road193.png': 'square', './Dataset/blue_squares/road123.png': 'square', './Dataset/blue_squares/road125.png': 'square', './Dataset/blue_squares/road127.png': 'square', './Dataset/stop_signs/road57.png': 'stop_sign', './Dataset/stop_signs/road52.png': 'stop_sign', './Dataset/stop_signs/road60.png': 'stop_sign', './Dataset/stop_signs/road62.png': 'stop_sign', './Dataset/stop_signs/road59.png': 'stop_sign', './Dataset/stop_signs/road58.png': 'stop_sign', './Dataset/stop_signs/road70.png': 'stop_sign', './Dataset/stop_si

### Finding a perfect circle

By using the HoughCircles function we are able to detect perfect circles in an image.

There are some cases where it fits very well but if the sign is slanted no decent circle can be found

In [37]:
circles =  cv.HoughCircles(gray_blurred, cv.HOUGH_GRADIENT, 1.5, 100)
output = img.copy()
# ensure at least some circles were found
print(circles)
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circles:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv.circle(output, (x, y), r, (0, 255, 0), 4)
        cv.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1) #center of circle
        # show the output image
    cv.imshow("output", np.hstack([img, output]))

cv.waitKey(0)
cv.destroyAllWindows()

NameError: name 'gray_blurred' is not defined

### Commented out cell

To start we need to collect all of the xml files and the corresponding png files so we can parse them.
Our dataset has 877 pictures of traffic sings with 4 distinctions: 'trafficlight', 'speedlimit', 'crosswalk' and 'stop'.

In [ ]:
# xml_files = [name for name in os.listdir('./Dataset/annotations')]
# validation_dictionary = {}

# for name in xml_files:
#     # Parse the xml
#     mytree = ET.parse('./Dataset/annotations/' + name)
    
#     # Get number from name
#     order = ''.join(i for i in name if i.isdigit())
    
#     # Get sign type
#     validation_dictionary[order] = mytree.getroot().find('object').find('name').text

# sign_types = []
# for entry in validation_dictionary:
#     if validation_dictionary[entry] not in sign_types:
#         sign_types.append(validation_dictionary[entry])
# print(sign_types)
# print(validation_dictionary)